<a href="https://colab.research.google.com/github/iamRahulB/DistilBERT-Sentiment-Analysis-Fine-tuned-Model-/blob/main/notebooke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'twitter-sentiment-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1340873%2F2231927%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240224%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240224T144405Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D52045d58ef106381252c6e208a75a2082e0381290b967c3ce9d9870b1388c64f9ea277d499c14458c442eed8b16aa19979c2881205a8f7c19a9c19ce6a04dd8a30102d9e0828c9640c9400fc5660af28fe2555e70f8bb79a13ebcceb8ab634d8bffc80f039da456def967e8b7dbeff70b04749359c433ff4d4c2ac167a2dec7051df756c97f485a7154f9b8e6546f80ae5d5c8b28dfc5be5854e43edfeebc4b050919d997ecc3ca4bb79ccb5b0bd7eeb1ad9ccd34b2bd0a34d3be93057f446cdd52bf1808305441c8c0ab19a6dd9ea460cea82d02fb0ac4e9e527f5cbe8767ee157c31f09682d25c7af1b0f0d56b0f46ab3575ef5edae79f8ca70d96158f0436'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 7966522 bytes downloaded
Downloaded and uncompressed: twitter-sentiment-dataset
Data source import complete.


In [2]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [3]:
import transformers
from transformers import TFAutoModelForSequenceClassification,AutoTokenizer
import datasets
from datasets import Dataset,load_dataset
import pandas as pd



In [4]:
checkpoint="distilbert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=TFAutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [5]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955779 (255.42 MB)
Trainable params: 66955779 (255.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
df=pd.read_csv("/kaggle/input/twitter-sentiment-dataset/Twitter_Data.csv")

In [7]:
df.isnull().sum()

clean_text    4
category      7
dtype: int64

In [8]:
df['category'] = df['category'].replace({-1: 0, 0: 1, 1: 2})


In [9]:
df.dropna(inplace=True)

In [10]:
df.reset_index(drop=True, inplace=True)


In [11]:
# df["labels"]=df["labels"].astype("int64")

In [12]:
raw_data=Dataset.from_pandas(df)

In [13]:
raw_data=raw_data.train_test_split(test_size=0.2)

In [14]:
# raw_data["train"]["clean_text"][0:6]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162969 entries, 0 to 162968
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162969 non-null  object 
 1   category    162969 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [16]:
df.isnull().sum()

clean_text    0
category      0
dtype: int64

In [17]:
def token(example):
    inputs=example["clean_text"]
    targets=example["category"]
    model_inputs=tokenizer(inputs,truncation=True)
    model_inputs["labels"]=targets
    return model_inputs


In [18]:
# model_inputs=tokenizer(inputs,truncation=True,max_length=128)

In [19]:
raw_data=raw_data.map(token,batched=True)

Map:   0%|          | 0/130375 [00:00<?, ? examples/s]

Map:   0%|          | 0/32594 [00:00<?, ? examples/s]

In [20]:
raw_data

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 130375
    })
    test: Dataset({
        features: ['clean_text', 'category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 32594
    })
})

In [21]:
from transformers import DataCollatorWithPadding


In [22]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [23]:
tf_train = model.prepare_tf_dataset(
    raw_data["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator,
)


In [24]:
tf_train

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(32, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(32, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(32,), dtype=tf.float32, name=None))>

In [25]:
# Importing the Adam optimizer from TensorFlow
from tensorflow.keras.optimizers import Adam

# Compiling the model with the Adam optimizer and a specified learning rate
model.compile(optimizer=Adam(3e-5))

In [26]:
model.fit(x=tf_train, epochs=2)

Epoch 1/2
4074/4074 [==============================] - 793s 189ms/step - loss: 0.1972
Epoch 2/2
4074/4074 [==============================] - 765s 188ms/step - loss: 0.0705


In [79]:
text="moview was very great bro"

In [80]:
text_tokened=tokenizer(text,truncation=True,return_tensors="tf")

In [81]:
logits=model(**text_tokened).logits

In [82]:
logits

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-3.3463454, -2.7396176,  5.8593926]], dtype=float32)>

In [83]:
class_labels={0:"negative",1:"neutral",2:"positive"}

In [84]:
import tensorflow as tf


probabilities = tf.nn.softmax(logits)

predicted_class_index = tf.argmax(probabilities, axis=-1)

class_labels = class_labels[int(predicted_class_index)]

print("Predicted class:", class_labels)

Predicted class label: positive
